In [ ]:
%cd /data/cmpe258-sp24/miaosen/CMPE249/HW_1/working

/fs/atipa/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working


In [ ]:
!ls

final_model_2.pth  final_model_3.pth  final_model.pth  Kitti


In [ ]:
import torch
import torchvision
print(torch.__version__)
print(torchvision.__version__)

2.4.1
0.19.1


In [ ]:
print(torch.cuda.is_available())
#print(torch.cuda.device_count())

True


In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
#creating the symlink for detection
!git clone https://github.com/lkk688/DeepDataMiningLearning.git
%cd DeepDataMiningLearning
!pip install flit
!flit install --symlink

In [ ]:
from DeepDataMiningLearning.detection.dataset_kitti import MyKittiDetection

In [ ]:
from tqdm import tqdm

In [ ]:
import DeepDataMiningLearning.detection.transforms as T
def get_transformsimple(train):
    transforms = []
    transforms.append(T.PILToTensor())
    transforms.append(T.ToDtype(torch.float, scale=True))
    # if train:
    #     transforms.append(RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [ ]:
rootPath='/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/Kitti'
is_train = True
kittidataset = MyKittiDetection(rootPath, train=True, transform=get_transformsimple(is_train))

In [ ]:
print(kittidataset.INSTANCE_CATEGORY_NAMES)

['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc', 'DontCare']


In [ ]:
print("Dataset len:", len(kittidataset))

Dataset len: 7481


In [ ]:
img, target = kittidataset[0]

In [ ]:
from DeepDataMiningLearning.detection import utils

In [ ]:
BATCH_SIZE=16
WORKERS = 16
train_collate_fn = utils.collate_fn
train_sampler = torch.utils.data.RandomSampler(kittidataset)
train_batch_sampler = torch.utils.data.BatchSampler(train_sampler, BATCH_SIZE, drop_last=True)

In [ ]:
data_loader = torch.utils.data.DataLoader(
        kittidataset, batch_sampler=train_batch_sampler, num_workers=WORKERS, collate_fn=train_collate_fn
    )

In [ ]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# load a model pre-trained pre-trained
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

# replace the classifier with a new one, that has
# num_classes which is user-defined
num_classes = 10  # 1 class (person) + background
# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features
# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

# move model to the right device
model.to(device)

/data/cmpe258-sp24/miaosen/miniconda3/envs/CMPE249py311/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/data/cmpe258-sp24/miaosen/miniconda3/envs/CMPE249py311/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [ ]:
save_path = '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_3.pth'

In [ ]:
model.load_state_dict(torch.load(save_path, map_location=torch.device(device)))

/tmp/ipykernel_27451/2812834572.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(save_path, map_location=torch.device('cpu')))


<All keys matched successfully>

In [ ]:
save_path = '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_3.pth'

In [ ]:
# Define optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Number of epochs
num_epochs = 25
i = 0

for epoch in tqdm(range(num_epochs), desc="Processing"):
    model.train()
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {losses.item()}")
    lr_scheduler.step()
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to '{save_path}'")

Processing:   0%|                                                                                     | 0/25 [00:00<?, ?it/s]

Epoch: 1/25, Loss: 0.4099661111831665


Processing:   4%|██▉                                                                       | 1/25 [12:02<4:49:03, 722.65s/it]

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_3.pth'
Epoch: 2/25, Loss: 0.3676679730415344


Processing:   8%|█████▉                                                                    | 2/25 [24:01<4:36:09, 720.43s/it]

...

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_3.pth'
Epoch: 18/25, Loss: 0.33914101123809814


Processing:  72%|███████████████████████████████████████████████████                    | 18/25 [3:35:50<1:23:56, 719.53s/it]

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_3.pth'
Epoch: 19/25, Loss: 0.33043575286865234


Processing:  76%|█████████████████████████████████████████████████████▉                 | 19/25 [3:47:50<1:11:58, 719.77s/it]

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_3.pth'


In [ ]:
# Define optimizer
params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)

# Learning rate scheduler
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

# Number of epochs
num_epochs = 25-19
i = 0

for epoch in tqdm(range(num_epochs), desc="Processing"):
    model.train()
    for images, targets in data_loader:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

    print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {losses.item()}")
    lr_scheduler.step()
    torch.save(model.state_dict(), save_path)
    print(f"Model saved to '{save_path}'")

Processing:   0%|                                                                                      | 0/6 [00:00<?, ?it/s]

Epoch: 1/6, Loss: 0.3237442374229431


Processing:  17%|████████████▊                                                                | 1/6 [11:58<59:52, 718.43s/it]

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_3.pth'
Epoch: 2/6, Loss: 0.3392223119735718


Processing:  33%|█████████████████████████▋                                                   | 2/6 [23:57<47:55, 718.91s/it]

...

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_3.pth'
Epoch: 5/6, Loss: 0.2320021241903305


Processing:  83%|████████████████████████████████████████████████████████████████▏            | 5/6 [59:57<11:59, 719.70s/it]

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_3.pth'
Epoch: 6/6, Loss: 0.21874189376831055


Processing: 100%|███████████████████████████████████████████████████████████████████████████| 6/6 [1:11:57<00:00, 719.64s/it]

Model saved to '/data/cmpe258-sp24/miaosen/CMPE249/HW_1/working/final_model_3.pth'


In [ ]:
training_loss = [0.5486985445022583, 0.5177141427993774, 0.3774336874485016, 0.2998450696468353, 0.4477429986000061, 0.42523491382598877,
                0.28533148765563965, 0.3745960295200348, 0.3087141215801239, 0.41189268231391907, 0.4973844289779663, 0.39810168743133545,
                0.44491779804229736, 0.4469603896141052, 0.32026392221450806, 0.3636575937271118, 0.33206406235694885, 0.4269688129425049,
                0.44354248046875, 0.4131874442100525, 0.32151132822036743, 0.34521880745887756, 0.3112267851829529, 0.398506224155426,
                0.38528257608413696, 0.39250510931015015, 0.42429405450820923, 0.4108785092830658, 0.3676679730415344, 0.40374472737312317,
                0.3187902569770813, 0.30164363980293274, 0.27550289034843445, 0.24123072624206543, 0.35506439208984375, 0.23902638256549835,
                0.3400615155696869, 0.367064893245697, 0.3497505784034729, 0.2871348559856415, 0.37905365228652954, 0.2433825433254242, 0.2944604754447937,
                0.27428725361824036, 0.33914101123809814, 0.33043575286865234, 0.3392223119735718, 0.3051171898841858, 0.267776757478714, 0.2320021241903305,
                0.21874189376831055 ]